In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

  # Cargar el dataset
data = pd.read_csv('FIS\dataset_wide.csv')

  # Mostrar las primeras filas del dataset
print(data.head())

          P         D          L          R P_fuzzy_set  P_membership  \
0 -0.000480  0.526110  -9.743533  63.120691          ok      0.998800   
1  0.944574  0.025943 -38.727943  -5.020554   muy cerca      0.907624   
2  0.023568  0.013238  12.355830  36.212792          ok      0.941080   
3 -0.982377  0.996781  62.889400  -7.795086   muy lejos      0.970628   
4  0.382476 -0.454730  13.321278  40.839750       cerca      0.956189   

   D_fuzzy_set  D_membership                     L_fuzzy_set  L_membership  \
0  acercandose      1.000000           muy lento hacia atrás      0.987177   
1   sin cambio      0.870285  más o menos rápido hacia atrás      0.872794   
2   sin cambio      0.933810         muy lento hacia delante      0.882208   
3  acercandose      1.000000       rápidamente hacia delante      0.855530   
4   alejandose      1.000000         muy lento hacia delante      0.833936   

                  R_fuzzy_set  R_membership  \
0   rápidamente hacia delante      0.843965  

In [2]:
X = data[['P','D','L','R',]]
y = data['CAT']
z = data[['P_fuzzy_set', 'D_fuzzy_set', 'L_fuzzy_set', 'R_fuzzy_set']].apply(lambda x: ' '.join(x), axis=1)

In [4]:
with open('FIS\corpus.txt', 'r', encoding='utf-8') as f:
    contenido = f.read()

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts([contenido])
num_decoder_tokens = len(tokenizer.word_index) + 1
y_tokenized = tokenizer.texts_to_sequences(y)

In [6]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_tokenized, test_size=0.2, random_state=42)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Obtener el índice de las filas de X_train, X_val y X_test
idx_train = X_train.index
idx_val = X_val.index
idx_test = X_test.index

In [7]:
# Dividir z en z_train, z_val y z_test
z_train = z.loc[idx_train].values
z_val = z.loc[idx_val].values
z_test = z.loc[idx_test].values

In [8]:
z_train = tokenizer.texts_to_sequences(z_train)
z_val = tokenizer.texts_to_sequences(z_val)
z_test = tokenizer.texts_to_sequences(z_test)

In [9]:
X_train = X_train[['P','D','L','R']].values
X_val = X_val[['P','D','L','R']].values
X_test = X_test[['P','D','L','R']].values

In [10]:
from sklearn.preprocessing import StandardScaler

# Ajusta las columnas numéricas con StandardScaler
scaler_values = StandardScaler()

scaler_values.fit(X_train)

StandardScaler()

In [11]:
X_train_scaled = scaler_values.transform(X_train)
X_val_scaled =scaler_values.transform(X_val)
X_test_scaled =scaler_values.transform(X_test)

In [12]:
max_len = max(len(s) for s in y_tokenized)

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


y_train = pad_sequences(y_train, maxlen=num_decoder_tokens, padding='post', truncating='post')
y_val = pad_sequences(y_val, maxlen=num_decoder_tokens, padding='post', truncating='post')
y_test = pad_sequences(y_test, maxlen=num_decoder_tokens, padding='post', truncating='post')
z_train = pad_sequences(z_train, maxlen=num_decoder_tokens, padding='post', truncating='post')
z_val = pad_sequences(z_val, maxlen=num_decoder_tokens, padding='post', truncating='post')
z_test = pad_sequences(z_test, maxlen=num_decoder_tokens, padding='post', truncating='post')

In [14]:
print('tamaño X train_scaled', X_train_scaled.shape)
print('tamaño y train', y_train.shape)
print('tamaño z train', z_train.shape)


print('tamaño X val_scaled', X_val_scaled.shape)
print('tamaño y val', y_val.shape)
print('tamaño z val_scaled', z_val.shape)

print('tamaño X test_scaled', X_test_scaled.shape)
print('tamaño y test', y_test.shape)
print('tamaño z test', z_test.shape)


tamaño X train_scaled (5999, 4)
tamaño y train (5999, 95)
tamaño z train (5999, 95)
tamaño X val_scaled (2000, 4)
tamaño y val (2000, 95)
tamaño z val_scaled (2000, 95)
tamaño X test_scaled (2000, 4)
tamaño y test (2000, 95)
tamaño z test (2000, 95)


In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Definir dimension de entrada para el encoder
encoder_inputs = Input(shape=(1, 4))

# Capa LSTM en el encoder
encoder_lstm = LSTM(128, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)

# Se descartan las salidas del encoder y solo se toman los estados
encoder_states = [state_h, state_c]

In [16]:
# Definir dimension de entrada para el decoder
decoder_inputs = Input(shape=(None,))

# Capa de embedding en el decoder
decoder_embedding = Embedding(num_decoder_tokens, output_dim=100)
decoder_inputs_embedded = decoder_embedding(decoder_inputs)

# Capa LSTM en el decoder, con los estados del encoder como inicialización
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_embedded, initial_state=encoder_states)

# Capa densa con activación softmax en el output
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Definir modelo encoder-decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilar el modelo
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [18]:
# Entrenar el modelo
model.fit([X_train_scaled.reshape(5999,1,4), z_train], 
    y_train.reshape(y_train.shape[0], y_train.shape[1], 1),
     batch_size=30, 
     epochs=15, 
     validation_data=([X_val_scaled.reshape(X_val.shape[0],1,4), z_val], y_val.reshape(y_val.shape[0], y_val.shape[1], 1)))

Epoch 1/15
200/200 [==============================] - 48s 241ms/step - loss: 0.8324 - val_loss: 0.8215
Epoch 2/15
200/200 [==============================] - 47s 236ms/step - loss: 0.7835 - val_loss: 0.7679
Epoch 3/15
200/200 [==============================] - 48s 241ms/step - loss: 0.7316 - val_loss: 0.6954
Epoch 4/15
200/200 [==============================] - 49s 247ms/step - loss: 0.6854 - val_loss: 0.6946
Epoch 5/15
200/200 [==============================] - 47s 236ms/step - loss: 0.6488 - val_loss: 0.6116
Epoch 6/15
200/200 [==============================] - 46s 231ms/step - loss: 0.6167 - val_loss: 0.6002
Epoch 7/15
200/200 [==============================] - 46s 229ms/step - loss: 0.5869 - val_loss: 0.6238
Epoch 8/15
200/200 [==============================] - 46s 232ms/step - loss: 0.5597 - val_loss: 0.5479
Epoch 9/15
169/200 [========================>.....] - ETA: 6s - loss: 0.5392

KeyboardInterrupt: 

In [17]:
model.evaluate([X_test.reshape(X_test.shape[0],1,4),z_test])

63/63 [==============================] - 1s 3ms/step - loss: 0.0000e+00


0.0

In [18]:
def predictions(preds):
    preds_words = []
    for i in range(preds.shape[0]):
        preds_row = preds[i]
        preds_row_words = []
        for idx in preds_row:
            if idx in tokenizer.index_word:
                preds_row_words.append(tokenizer.index_word[idx])
        preds_words.append(preds_row_words)

    return preds_words

In [19]:
inPrueba = np.array([[-0.997111302,0.430978894,59.95185503,-9.90371007]])
inPrueba = scaler_values.transform(inPrueba).reshape(1,1,4)

In [20]:
z_prueba = ['muy lejos acercandose muy rapidamente hacia delante muy lento hacia atrás']
z_prueba = tokenizer.texts_to_sequences(z_prueba)
z_prueba = pad_sequences(z_prueba, maxlen=max_len, padding='post', truncating='post')

In [21]:
inPrueba

array([[[-1.52516433,  0.84932278,  0.76718668, -0.47877844]]])

In [22]:
z_prueba

array([[ 9, 24,  9,  3, 22,  9, 29,  3, 12,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]])

In [23]:
preds = model.predict([inPrueba, np.array(z_prueba)])
preds = np.argmax(preds, axis=-1)

1/1 [==============================] - 1s 1s/step


In [24]:
frasesnlg = predictions(preds)
print(frasesnlg)

[['cuando', 'caso', 'muy', 'lejos', 'de', 'la', 'pared', 'pero', 'te', 'estás', 'estás', 'a', 'ella', 'gira', 'tu', 'rueda', 'izquierda', 'hacia', 'delante', 'delante', 'y', 'la', 'rueda', 'derecha', 'derecha', 'lento', 'atrás', 'atrás']]
